## Memory - 2



### Backed by a Vector Store

VectorStoreRetrieverMemory 内存存储在一个向量存储和查询最“突出”的顶 K 文档每次调用的记忆。

这与大多数其他 Memory 类的不同之处在于，它不显式地跟踪交互的顺序。

In [10]:
# 环境变量设置
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"

导入基础类库

In [11]:
from datetime import datetime
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain_core.prompts import PromptTemplate

#### 初始化向量存储并存储本地pdf

In [21]:
import faiss

from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader



# embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
# index = faiss.IndexFlatL2(embedding_size)
# embedding_fn = OpenAIEmbeddings().embed_query

loader = PyPDFLoader("test.pdf")
documents = loader.load_and_split()

vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings())
# vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

#### 创建 VectorStoreRetrieverMemory


In [22]:
# In actual usage, you would set `k` to be a higher value, but we use k=1 to show that
# the vector lookup still returns the semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# When added to an agent, the memory object can save pertinent information from conversations or used tools
memory.save_context({"input": "我最喜欢的食物是汉堡"}, {"output": "好的我知道了"})
memory.save_context({"input": "我最喜欢的运动是篮球"}, {"output": "..."})
memory.save_context({"input": "我不喜欢凯尔特人"}, {"output": "好的"}) #

In [23]:
print(memory.load_memory_variables({"prompt": "我最喜欢什么运动?"})["history"])

input: 我最喜欢的运动是篮球
output: ...


#### Chain中使用

In [24]:
llm = OpenAI(temperature=0) # Can be any valid LLM
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is Perry, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
易速鲜花集团 
 17  
 
 
第三章 规范用语  
 
一、早上上班时，员工与员工相遇，应微笑相互问候。规范用语：
“早上好！”。  
二、平常员工相遇时规范用语为“你好！”。  
三、员工若与上司相遇，员工先向上司问候，并在问候语前加上上
司的行政职务称谓。如：“某总，早上好！”“某经理，您好”  
四、迎宾用语：“早上好，欢迎光临！”  
五、送宾用语：“请慢走，欢迎下次光临！”  
六、园区遇到顾客：“您好，欢迎光临 易速鲜花 ” 
七、会议用语：  
会前：主持人 -----“各位同事早上好（下午好）”  
参会人员 ----“好” 击掌三下  
会后：主持人 -----“会议到此结束，谢谢！”  
会议期间参会人员一律将手机调成静音或关机。  
八、电话用语规范：

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, my name is Perry, what's up?
AI:

> Finished chain.


' Hello Perry, my name is AI. I am an artificial intelligence programmed to assist with various tasks. Currently, I am functioning as a virtual assistant for the company 易速鲜花集团. How can I help you?'

查询加载pdf的数据

In [27]:
conversation_with_summary.predict(input="易速鲜花员工的个人卫生要求是什么?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
易速鲜花集团 
 12 第二节  仪容仪表规范  
 
一、 仪  容 
1. 发型要求  
头发要梳理整齐，男员工头发长度应适中，前不过眉、后不过
领、侧不过耳、呈现自然色；女员工头发长度不可过肩，过肩发需
束起来，不可散落，并用深色且式样简洁，大小不超过十公分的头
饰，呈现自然色。  
2. 面部要求  
① 男员工不留胡须，并保持脸部干净整洁；女员工须着淡妆
（至少涂有口红），但不可浓妆艳抹。  
② 眼睛近视的员工尽量配戴隐形眼镜，如必须配戴近视眼镜
的则眼镜款式要简单、大方。  
3. 手部要求  
① 指甲修剪整 齐、保持清洁，不得留长指甲。不可涂有色指
甲油。 
② 除了样式简洁的手表、结婚戒指、耳环（简单的、小的、
不晃动的耳环）外，不得在穿着制服时佩戴其它饰品。  
二、 仪  表 
1. 制服要求  
必须按规定穿着制服，保持整齐、清洁、熨烫平整、无破损，

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: 易速鲜花员工的个人卫生要求是什么?
AI:

> Finished chain.


' 易速鲜花员工的个人卫生要求包括仪容仪表规范和制服要求。具体来说，发型要求是头发要梳理整齐，男员工头发长度应适中，前不过眉、后不过领、侧不过耳、呈现自然色；女员工头发长度不可过肩，过肩发需束起来，不可散落，并用深色且式样简洁，大小不超过十公分的头饰，呈现自然色。面部要求是男员工不留胡须，并保持脸部干净整洁；女员工须着淡妆（至少涂有口红），但不可浓妆艳抹。眼睛近视的员工尽量配戴隐形眼镜，如必须配戴近视眼镜'

In [16]:
# Here, the basketball related content is surfaced
conversation_with_summary.predict(input="我最喜欢的运动是什么?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: 我最喜欢的运动是篮球
output: ...

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: 我最喜欢的运动是什么?
AI:

> Finished chain.


' 您最喜欢的运动是篮球。我可以告诉您关于篮球的一些有趣的事实。篮球是一种流行的团队运动，通常由两支队伍各有五名球员组成。它的目标是将球投入对方的篮筐，得分最多的一方获胜。篮球起源于美国，现在已经成为世界各地的流行运动。它也是一项奥运会正式比赛项目。您知道吗，篮球最初是用一个桃子篮和一个足球来玩的吗？后来，随着发展，篮球变成了我们现在熟悉的形式。您还有什么想知道的吗？'

In [17]:
conversation_with_summary.predict(input="我最喜欢的食物是什么?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: 我最喜欢的食物是汉堡
output: 好的我知道了

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: 我最喜欢的食物是什么?
AI:

> Finished chain.


' 您最喜欢的食物是什么？我可以告诉您，您之前告诉我您最喜欢的食物是汉堡。您喜欢的食物是汉堡，对吗？'

In [19]:
conversation_with_summary.predict(input="我的名字是什么?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: What's my name?
response:  Your name is Perry. You introduced yourself to me earlier. Did you forget?

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: 我的名字是什么?
AI:

> Finished chain.


' 你的名字是佩里。你之前向我介绍过自己。你忘记了吗？'